**NOMBRE:**
**Sergio Rodrigo Cárdenas Rivera**

Poniendo en practica lo visto recientemente

**Ejercicio 1.Consolidación y Limpieza de Datos de Ventas**

Tienen los siguientes datasets:

ventas_mes_1: Contiene las ventas de productos para el primer mes.

ventas_mes_2: Contiene las ventas para el segundo mes.

productos_info: Contiene información adicional sobre los productos (categoría y precio por unidad).

Tareas:

1. Usa concat para combinar los datasets ventas_mes_1 y ventas_mes_2.
2. Detecta y elimina outliers en la columna cantidad_vendida (criterio:valores fuera de $Q_1-1.5xIQR$ y $Q_3+1.5xIQR$).
3. Usa merge para combinar el dataset consolidado con productos_info, usando el ID_producto como clave.
4. Usa combine_first para completar valores faltantes en la columna precio_unitario del dataset resultante con un dataset auxiliar llamado precios_backup.

In [1]:
import pandas as pd
import numpy as np

# Ventas del primer mes
ventas_mes_1 = pd.DataFrame({
    'ID_producto': [101, 102, 103, 104],
    'cantidad_vendida': [50, 200, 10, 100]
})

# Ventas del segundo mes
ventas_mes_2 = pd.DataFrame({
    'ID_producto': [101, 102, 103, 105],
    'cantidad_vendida': [55, 210, 15, 5]
})

# Información de productos
productos_info = pd.DataFrame({
    'ID_producto': [101, 102, 103, 104],
    'categoria': ['A', 'B', 'C', 'A'],
    'precio_unitario': [10.5, 20.0, np.nan, 15.0]
})

# Backup de precios
precios_backup = pd.DataFrame({
    'ID_producto': [103, 105],
    'precio_unitario': [18.0, 12.0]
})


In [2]:
#1 concat ventas_mes_1, ventas_mes_2
ventas = pd.concat([ventas_mes_1.iloc[:,:], ventas_mes_2.iloc[:, 1]], axis=1)
# ventas.rename(columns={"cantidad_vendida": "mes_1"})
ventas

,ID_producto,cantidad_vendida,cantidad_vendida
0,101,50,55
1,102,200,210
2,103,10,15
3,104,100,5


In [3]:
#Para calcular Q1, Q3 TQR:
def calcular_outlier(dataframe, columname):
  Q1 = dataframe[columname].quantile(0.25) # Primer cuartil
  Q3 = dataframe[columname].quantile(0.75) # Tercer cuartil
  IQR = Q3 - Q1
  lim_inf = Q1 - 1.5 * IQR
  lim_sup = Q3 + 1.5 * IQR
  return lim_inf, lim_sup

In [4]:
#2 
li, ls = calcular_outlier(ventas_mes_1, 'cantidad_vendida')

ventas_mes_1 = ventas_mes_1[ventas_mes_1['cantidad_vendida']<ls]
ventas_mes_1 = ventas_mes_1[ventas_mes_1['cantidad_vendida']>li]
ventas_mes_1

,ID_producto,cantidad_vendida
0,101,50
1,102,200
2,103,10
3,104,100


In [5]:
li, ls = calcular_outlier(ventas_mes_2, 'cantidad_vendida')

ventas_mes_2 = ventas_mes_2[ventas_mes_2['cantidad_vendida']<ls]
ventas_mes_2 = ventas_mes_2[ventas_mes_2['cantidad_vendida']>li]
ventas_mes_2

,ID_producto,cantidad_vendida
0,101,55
1,102,210
2,103,15
3,105,5


In [6]:
#3 merge datasets
# ventas_mes_1, ventas_mes_2
# ventas_det = pd.DataFrame(
#   {
#     'ID_producto': ventas_mes_1.iloc[:, 0],
#     'venta_mes_1': ventas_mes_1.iloc[:, 1],
#     'venta_mes_2': ventas_mes_2.iloc[:, 1],
#   }
# )
# pd.merge(ventas_det,productos_info, on='ID_producto')

ventas_det = pd.merge(ventas_mes_1, ventas_mes_2, on='ID_producto', how='outer')
ventas_det = pd.merge(ventas_det,productos_info, on='ID_producto', how='outer')

ventas_det = ventas_det.rename(columns={'cantidad_vendida_x': 'venta_mes_1','cantidad_vendida_y': 'venta_mes_2'})
ventas_det = ventas_det[['ID_producto', 'categoria', 'precio_unitario', 'venta_mes_1', 'venta_mes_2']]
ventas_det

,ID_producto,categoria,precio_unitario,venta_mes_1,venta_mes_2
0,101,A,10.5,50.0,55.0
1,102,B,20.0,200.0,210.0
2,103,C,NaN,10.0,15.0
3,104,A,15.0,100.0,NaN
4,105,NaN,NaN,NaN,5.0


In [7]:
#4
precios_backup.set_index('ID_producto', inplace=True)
ventas_det.set_index('ID_producto', inplace=True)
ventas_det.combine_first(precios_backup)
ventas_det

,categoria,precio_unitario,venta_mes_1,venta_mes_2
ID_producto,,,,
101,A,10.5,50.0,55.0
102,B,20.0,200.0,210.0
103,C,NaN,10.0,15.0
104,A,15.0,100.0,NaN
105,NaN,NaN,NaN,5.0


**Ejercicio 2: Datos Climáticos**

Tienes tres datasets relacionados con datos climáticos:

clima_dia: Registra las temperaturas y precipitaciones diarias.

promedio_mensual: Contiene los promedios mensuales históricos de temperatura y precipitación.

info_region: Proporciona información adicional sobre las regiones (región y altitud).

**Tareas:**
1. Usa merge para unir clima_dia con info_region utilizando la columna region como clave.
2. Usa concat para apilar los datos de promedio_mensual al resultado anterior, asegurándote de manejar los índices correctamente.
3. Detecta y elimina outliers en las columnas temperatura y precipitacion (utiliza la regla de $Q_1-1.5xIQR$ y $Q3+1.5xIQR$)
4. Usa combine_first para completar valores faltantes de precipitación utilizando un dataset auxiliar llamado precipitacion_backup.

In [8]:
# Clima diario
clima_dia = pd.DataFrame({
    'fecha': ['2024-01-01', '2024-01-02', '2024-01-03'],
    'region': ['Norte', 'Sur', 'Norte'],
    'temperatura': [35, np.nan, 40],
    'precipitacion': [10, 50, np.nan]
})

# Promedio mensual histórico
promedio_mensual = pd.DataFrame({
    'mes': ['Enero', 'Febrero'],
    'temperatura_media': [32, 28],
    'precipitacion_media': [20, 15]
})

# Información de regiones
info_region = pd.DataFrame({
    'region': ['Norte', 'Sur', 'Centro'],
    'altitud': [500, 300, 700]
})

# Backup de precipitación
precipitacion_backup = pd.DataFrame({
    'fecha': ['2024-01-03'],
    'precipitacion': [12]
})


In [9]:
# 1 merge
data_merged = pd.merge(clima_dia, info_region, on='region')
data_merged


,fecha,region,temperatura,precipitacion,altitud
0,2024-01-01,Norte,35.0,10.0,500
1,2024-01-02,Sur,NaN,50.0,300
2,2024-01-03,Norte,40.0,NaN,500


In [10]:
def get_month_literal_esp(date):
  months = [
    "enero", "febrero", "marzo", "abril", "mayo", "junio", 
    "julio", "agosto", "septiembre", "octubre", "noviembre", "diciembre" 
  ]
  month = int(date.split('-')[-2])
  return months[month-1].capitalize()

In [11]:
#2 concat
#Usa concat para apilar los datos de promedio_mensual al resultado anterior, asegurándote de manejar los índices correctamente.
# data_merged['mes'] = get_month_literal_esp(data_merged['fecha'])
data_merged['mes']=  data_merged['fecha'].apply(get_month_literal_esp)
dm = pd.concat([promedio_mensual, data_merged ])
dm


,mes,temperatura_media,precipitacion_media,fecha,region,temperatura,precipitacion,altitud
0,Enero,32.0,20.0,NaN,NaN,NaN,NaN,NaN
1,Febrero,28.0,15.0,NaN,NaN,NaN,NaN,NaN
0,Enero,NaN,NaN,2024-01-01,Norte,35.0,10.0,500.0
1,Enero,NaN,NaN,2024-01-02,Sur,NaN,50.0,300.0
2,Enero,NaN,NaN,2024-01-03,Norte,40.0,NaN,500.0


In [12]:
#3 delete outliers
# Detecta y elimina outliers en las columnas temperatura y precipitacion (utiliza la regla de $Q_1-1.5xIQR$ y $Q3+1.5xIQR$)
original_copy_dm = dm.copy()

li_temp, ls_temp = calcular_outlier(dm, 'temperatura')

li_prec, ls_prec = calcular_outlier(dm, 'precipitacion')

dm['temperatura'] = dm['temperatura'].fillna(dm['temperatura'].mean())
dm['precipitacion'] = dm['precipitacion'].fillna(dm['precipitacion'].mean())

dm = dm[dm['temperatura']<ls_temp]
dm = dm[dm['temperatura']>li_temp]

dm = dm[dm['temperatura']<ls_prec]
dm = dm[dm['temperatura']>li_prec]

dm

,mes,temperatura_media,precipitacion_media,fecha,region,temperatura,precipitacion,altitud
0,Enero,32.0,20.0,NaN,NaN,37.5,30.0,NaN
1,Febrero,28.0,15.0,NaN,NaN,37.5,30.0,NaN
0,Enero,NaN,NaN,2024-01-01,Norte,35.0,10.0,500.0
1,Enero,NaN,NaN,2024-01-02,Sur,37.5,50.0,300.0
2,Enero,NaN,NaN,2024-01-03,Norte,40.0,30.0,500.0


In [ ]:
#4 usa combine_first 
# Usa combine_first para completar valores faltantes de precipitación utilizando un dataset auxiliar llamado precipitacion_backup.

original_copy_dm.set_index('fecha').combine_first(
  precipitacion_backup.set_index('fecha')
)

,altitud,mes,precipitacion,precipitacion_media,region,temperatura,temperatura_media
fecha,,,,,,,
2024-01-01,500.0,Enero,10.0,NaN,Norte,35.0,NaN
2024-01-02,300.0,Enero,50.0,NaN,Sur,NaN,NaN
2024-01-03,500.0,Enero,12.0,NaN,Norte,40.0,NaN
NaN,NaN,Enero,NaN,20.0,NaN,NaN,32.0
NaN,NaN,Febrero,NaN,15.0,NaN,NaN,28.0


**Ejercicio 3: Análisis de Inventario**

Tienes dos datasets:

inventario_almacen_1: Contiene datos de inventario del almacén 1.

inventario_almacen_2: Contiene datos de inventario del almacén 2.

**Tareas:**
1. Usa concat para apilar los inventarios de ambos almacenes.
2. Detecta filas duplicadas y elimínalas.
3. Identifica outliers en la columna cantidad (criterio: valores fuera de $Q_1-1.5xIQR$ y $Q3+1.5xIQR$
4. Usa merge para agregar una columna de costo_unitario basada en un dataset auxiliar llamado costos.
5. Calcula el valor total del inventario (cantidad \times costo_unitario) y agrega este cálculo como una nueva columna.

In [21]:
# Inventario Almacén 1
inventario_almacen_1 = pd.DataFrame({
    'ID_producto': [1, 2, 3],
    'cantidad': [100, 200, 150],
    'almacen': ['A1', 'A1', 'A1']
})

# Inventario Almacén 2
inventario_almacen_2 = pd.DataFrame({
    'ID_producto': [2, 3, 4],
    'cantidad': [180, 150, 50],
    'almacen': ['A2', 'A2', 'A2']
})

# Costos por producto
costos = pd.DataFrame({
    'ID_producto': [1, 2, 3, 4],
    'costo_unitario': [10, 15, 20, 25]
})


In [24]:
#1 concat 
alm = pd.concat([inventario_almacen_1, inventario_almacen_2]).reset_index(drop=True)
alm

,ID_producto,cantidad,almacen
0,1,100,A1
1,2,200,A1
2,3,150,A1
3,2,180,A2
4,3,150,A2
5,4,50,A2


In [31]:
#2
alm.drop_duplicates(subset=['ID_producto'])

,ID_producto,cantidad,almacen
0,1,100,A1
1,2,200,A1
2,3,150,A1
5,4,50,A2


In [32]:
#3
li_cant, ls_cant = calcular_outlier(alm, 'cantidad')
a = alm[alm['cantidad']<ls_temp]
a = alm[alm['cantidad']>li_temp]
a

,ID_producto,cantidad,almacen
0,1,100,A1
1,2,200,A1
2,3,150,A1
3,2,180,A2
4,3,150,A2
5,4,50,A2


In [34]:
#4
costos = pd.merge(alm, costos, on='ID_producto')
costos


,ID_producto,cantidad,almacen,costo_unitario
0,1,100,A1,10
1,2,200,A1,15
2,3,150,A1,20
3,2,180,A2,15
4,3,150,A2,20
5,4,50,A2,25


In [35]:
#5
def calcular_precio_inv(a, b):
  return a*b

costos['costo_inventario'] = costos['cantidad'] * costos['costo_unitario']
costos

,ID_producto,cantidad,almacen,costo_unitario,costo_inventario
0,1,100,A1,10,1000
1,2,200,A1,15,3000
2,3,150,A1,20,3000
3,2,180,A2,15,2700
4,3,150,A2,20,3000
5,4,50,A2,25,1250
